In [14]:
import pandas as pd
import numpy as np
import sklearn as sk

In [15]:
df = pd.read_csv('Data/ListaCobroDetalle2025.csv')
# Ordernar por idCredito
df = df.sort_values(by=['idCredito'])
# Eliminar la fecha de cobro banco
df = df.drop(columns=['fechaCobroBanco'])
# Rellenar los nulos con 0
df = df.fillna(0)
print (df.shape)
df.head()

(2114172, 8)


,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,idRespuestaBanco
27854,156277,9872,41468078,2,969.2,969.2,0.0,4.0
112144,156458,9872,41558280,2,969.2,969.2,0.0,4.0
223989,156727,9872,41691792,2,484.6,484.6,0.0,4.0
160606,156566,9872,41613223,2,969.2,969.2,0.0,4.0
221225,156711,9872,41688488,2,484.6,484.6,0.0,4.0


In [16]:
# Para cada idCredito, restar el consecutivoCobro mínimo de ese grupo para que el orden inicie en 0

# Enumerar en base al índice del valor para cada idCredito
df['orden'] = df.groupby('idCredito').cumcount()
print(df.shape)
print(df.head(10))


(2114172, 9)
        idListaCobro  idCredito  consecutivoCobro  idBanco  montoExigible  \
27854         156277       9872          41468078        2          969.2   
112144        156458       9872          41558280        2          969.2   
223989        156727       9872          41691792        2          484.6   
160606        156566       9872          41613223        2          969.2   
221225        156711       9872          41688488        2          484.6   
195200        156625       9872          41656948        2          484.6   
142186        156525       9872          41593527        2          969.2   
216330        156692       9872          41684150        2          484.6   
181888        156602       9872          41641011        2          969.2   
80002         156378       9872          41524210        2          969.2   

        montoCobrar  montoCobrado  idRespuestaBanco  orden  
27854         969.2           0.0               4.0      0  
112144        969

In [17]:
# Filtra los valores a los 4 mas populares y si no esta dentro de esos 4, lo pone como otro o 1
top4_bancos = df['idBanco'].value_counts().nlargest(4).index
print (top4_bancos) # son 12, 2, 72, 21
# Hacer un diccionario de los 4 bancos mas populares de 0 a 3
# y el resto como 4
bancos = {12: 0, 2: 1, 72: 2, 21: 3}
# Agregamos el resto como 4
bancos.update({x: 4 for x in df['idBanco'].unique() if x not in bancos})
df['idBanco'] = df['idBanco'].apply(lambda x: x if x in top4_bancos else 1)
# Ahora la pasamos a dummy
df = pd.get_dummies(df, columns=['idBanco'], prefix='banco')



Index([12, 2, 72, 21], dtype='int64', name='idBanco')


In [18]:
print (df.head())
print (df.shape)

        idListaCobro  idCredito  consecutivoCobro  montoExigible  montoCobrar  \
27854         156277       9872          41468078          969.2        969.2   
112144        156458       9872          41558280          969.2        969.2   
223989        156727       9872          41691792          484.6        484.6   
160606        156566       9872          41613223          969.2        969.2   
221225        156711       9872          41688488          484.6        484.6   

        montoCobrado  idRespuestaBanco  orden  banco_1  banco_2  banco_12  \
27854            0.0               4.0      0    False     True     False   
112144           0.0               4.0      1    False     True     False   
223989           0.0               4.0      2    False     True     False   
160606           0.0               4.0      3    False     True     False   
221225           0.0               4.0      4    False     True     False   

        banco_21  banco_72  
27854      False     

In [19]:
# Para el input del modelo, primero necesitamos agrupar por idCredito y luego el ultimo valor de cada grupo es el output
Cobradores = df.groupby('idCredito')
# Cuenta cuantos cobros se han hecho
print (Cobradores.size())
print (len(Cobradores))
# Ver el maximo y el minimo de cada grupo en tamano
print (Cobradores.size().max())
print (Cobradores.size().min())

idCredito
9872      18
10983      3
30466     18
31375      2
33591      3
          ..
755683     1
755695     1
755726     1
755859    16
756595     5
Length: 28602, dtype: int64
28602
1116
1


In [20]:
# Creamos una lista donde cada elemento es el historial de un grupo como lista de listas (filas)
historial_por_grupo = [grupo.values.tolist() for _, grupo in Cobradores]
print (len(historial_por_grupo))
print (historial_por_grupo[0])

28602
[[156277, 9872, 41468078, 969.2, 969.2, 0.0, 4.0, 0, False, True, False, False, False], [156458, 9872, 41558280, 969.2, 969.2, 0.0, 4.0, 1, False, True, False, False, False], [156727, 9872, 41691792, 484.6, 484.6, 0.0, 4.0, 2, False, True, False, False, False], [156566, 9872, 41613223, 969.2, 969.2, 0.0, 4.0, 3, False, True, False, False, False], [156711, 9872, 41688488, 484.6, 484.6, 0.0, 4.0, 4, False, True, False, False, False], [156625, 9872, 41656948, 484.6, 484.6, 0.0, 4.0, 5, False, True, False, False, False], [156525, 9872, 41593527, 969.2, 969.2, 0.0, 4.0, 6, False, True, False, False, False], [156692, 9872, 41684150, 484.6, 484.6, 0.0, 4.0, 7, False, True, False, False, False], [156602, 9872, 41641011, 969.2, 969.2, 0.0, 4.0, 8, False, True, False, False, False], [156378, 9872, 41524210, 969.2, 969.2, 0.0, 4.0, 9, False, True, False, False, False], [156311, 9872, 41483305, 969.2, 969.2, 0.0, 4.0, 10, False, True, False, False, False], [156749, 9872, 41702921, 484.6, 484

In [21]:
for i in range(len(historial_por_grupo)):
    print (len(historial_por_grupo[i]))

18
3
18
2
3
10
18
11
8
18
18
36
1
18
18
1
534
660
1
33
1080
1
18
170
2
711
1
1
123
1
3
249
204
18
2
11
1
6
3
24
18
22
143
7
1
716
1
159
36
18
7
23
18
19
2
1
27
218
18
18
1
18
28
706
8
1
22
1
18
243
648
1
18
18
3
1
33
3
2
18
23
18
3
6
243
1
1
8
1080
2
1
1
3
249
1
218
8
1
509
39
1
3
920
966
1
711
692
3
36
716
159
636
1
143
18
18
6
8
18
18
224
1
1
9
716
18
1
3
18
1
25
1
18
18
144
24
3
1
42
297
18
18
18
18
18
23
1
12
3
18
698
7
18
6
2
18
18
8
650
18
1
3
1
18
18
3
304
1
42
2
36
1
1
3
18
18
1
18
23
1
12
6
1
18
1008
6
18
1
1
18
18
18
3
1
18
18
3
2
3
18
4
24
3
42
1
1
6
4
18
18
1
12
18
18
36
1
68
18
2
18
1
18
1
1
1
3
2
701
18
18
18
18
18
18
383
42
2
18
18
18
6
18
1
6
1
1
1
18
31
1
42
18
8
18
12
18
18
18
18
68
18
12
155
36
18
18
1
8
1
18
18
18
18
18
1
702
1
18
1
18
1
18
18
18
53
18
18
18
18
8
18
2
18
1
18
4
12
1
9
6
18
23
1
18
8
4
56
2
147
18
31
42
11
6
1
1
18
6
1
1
18
1
12
1
1
2
18
18
68
12
18
18
716
18
18
7
18
18
655
19
340
18
37
18
716
1
1
1
112
12
1
12
1
3
18
6
9
9
18
2
23
698
18
1
18
18
383

In [22]:
# Calcula el valor de predicción para cada grupo: el último valor de idRespuestaBanco
# 0 = no hubo saldo, 4 = sí se hizo el cobro

predicciones = Cobradores['idRespuestaBanco'].last().reset_index()
predicciones.rename(columns={'idRespuestaBanco': 'valor_prediccion'}, inplace=True)
print(predicciones.head())

   idCredito  valor_prediccion
0       9872               4.0
1      10983               0.0
2      30466               4.0
3      31375               4.0
4      33591               4.0


In [23]:
# Veamos cuantas predicciones tenemos
print (predicciones['valor_prediccion'].value_counts(normalize=True))
# Aplicar un cambio, pasamos de 4 a 1, 26 a 1, y el resto a any a 0
dicccionario = {0: 1, 4: 0, 26: 1}
predicciones['valor_prediccion'] = predicciones['valor_prediccion'].replace(dicccionario)
# Ahora checamos para los que son cualqueir otro valor
predicciones['valor_prediccion'] = np.where(predicciones['valor_prediccion'].isin([0, 1]), predicciones['valor_prediccion'], 0)
# Ahora lo pasamos a 0 y 1
print (predicciones['valor_prediccion'].value_counts(normalize=True))

valor_prediccion
4.0     0.689567
0.0     0.248689
8.0     0.019614
26.0    0.014230
13.0    0.008985
3.0     0.007622
2.0     0.006783
1.0     0.003426
30.0    0.000699
88.0    0.000210
40.0    0.000070
36.0    0.000035
10.0    0.000035
6.0     0.000035
Name: proportion, dtype: float64
valor_prediccion
0.0    0.733655
1.0    0.266345
Name: proportion, dtype: float64


In [24]:
# Ahora visualizemos el inputs y el output del primer grupo
random = np.random.randint(0, len(historial_por_grupo))
# Visualiza el input (X_0:t-1), el último input (X_t) y el output (Y_t) del grupo seleccionado
X_0 = historial_por_grupo[random][:-1]  # Historial sin el último
X_t = historial_por_grupo[random][-1]  # Último input
Y_t = predicciones['valor_prediccion'][random]

# Print the length of the input and output
print("Longitud de X_0:t-1 (historial sin el último):", len(X_0))
print("Longitud de X_t (última fila del grupo):", len(X_0))


Longitud de X_0:t-1 (historial sin el último): 833
Longitud de X_t (última fila del grupo): 833


In [25]:
# Hacer el split de los datos en train, test y validación de {80, 10, 10}
# Primero separamos el input y el output
X = df.drop(columns=['idCredito', 'idRespuestaBanco'])
y = predicciones['valor_prediccion']
# Ahora hacemos el split
X_train, X_temp, y_train, y_temp = X[:int(0.8*len(X))], X[int(0.8*len(X)):], y[:int(0.8*len(X))], y[int(0.8*len(X)):]
X_val, X_test, y_val, y_test = X_temp[:int(0.5*len(X_temp))], X_temp[int(0.5*len(X_temp)):], y_temp[:int(0.5*len(y_temp))], y_temp[int(0.5*len(y_temp)):]
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)
print("Tamaño del conjunto de entrenamiento:", y_train.shape)
print("Tamaño del conjunto de validación:", y_val.shape)
print("Tamaño del conjunto de prueba:", y_test.shape)

Tamaño del conjunto de entrenamiento: (1691337, 11)
Tamaño del conjunto de validación: (211417, 11)
Tamaño del conjunto de prueba: (211418, 11)
Tamaño del conjunto de entrenamiento: (28602,)
Tamaño del conjunto de validación: (0,)
Tamaño del conjunto de prueba: (0,)


In [ ]:
class PreprocesadorCobros:
    def __init__(self, path_csv):
        self.path_csv = path_csv
        self.df = None
        self.Cobradores = None
        self.historial_por_grupo = None
        self.predicciones = None
        self.top4_bancos = None
        self.bancos = None

    # --- Carga y limpieza de datos ---
    def cargar_datos(self):
        self.df = pd.read_csv(self.path_csv)
        return self

    def ordenar_y_limpiar(self):
        self.df = self.df.sort_values(by=['idCredito'])
        if 'fechaCobroBanco' in self.df.columns:
            self.df = self.df.drop(columns=['fechaCobroBanco'])
        self.df = self.df.fillna(0)
        return self

    # --- Procesamiento de columnas ---
    def agregar_orden(self):
        self.df['orden'] = self.df.groupby('idCredito').cumcount()
        return self

    def procesar_bancos(self):
        self.top4_bancos = self.df['idBanco'].value_counts().nlargest(4).index
        self.bancos = {12: 0, 2: 1, 72: 2, 21: 3}
        self.bancos.update({x: 4 for x in self.df['idBanco'].unique() if x not in self.bancos})
        self.df['idBanco'] = self.df['idBanco'].apply(lambda x: x if x in self.top4_bancos else 1)
        self.df = pd.get_dummies(self.df, columns=['idBanco'], prefix='banco')
        return self

    # --- Agrupación y generación de features ---
    def agrupar_por_credito(self):
        self.Cobradores = self.df.groupby('idCredito')
        return self

    def crear_historial_por_grupo(self):
        self.historial_por_grupo = [grupo.values.tolist() for _, grupo in self.Cobradores]
        return self

    # --- Generación de etiquetas ---
    def calcular_predicciones(self):
        self.predicciones = self.Cobradores['idRespuestaBanco'].last().reset_index()
        self.predicciones.rename(columns={'idRespuestaBanco': 'valor_prediccion'}, inplace=True)
        dicccionario = {0: 1, 4: 0, 26: 1}
        self.predicciones['valor_prediccion'] = self.predicciones['valor_prediccion'].replace(dicccionario)
        self.predicciones['valor_prediccion'] = np.where(
            self.predicciones['valor_prediccion'].isin([0, 1]),
            self.predicciones['valor_prediccion'],
            0
        )
        return self

    # --- Ejecutar todo el preprocesamiento ---
    def ejecutar_todo(self):
        self.cargar_datos()
        self.ordenar_y_limpiar()
        self.agregar_orden()
        self.procesar_bancos()
        self.agrupar_por_credito()
        self.crear_historial_por_grupo()
        self.calcular_predicciones()
        return self.split_datos()